# **Data Science Capstone Project**
##*Segmenting and Clustering Neighborhoods in Toronto*

In [40]:
#Import libraries for webscraping in order to create Toronto Postal Code DataFrame
#!pip install bs4
#!pip install requests

from bs4 import BeautifulSoup # for webscraping.
import requests  # download webpage
import pandas as pd # Data manipulation

print('Libraies successfuly installed')

Libraies successfuly installed


In [41]:
#dowload Toronto postal code webpage table

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

#Scrape data from html tabla

# get the contents of the webpage in text format and store in a variable called toronto_data
toronto_data  = requests.get(url).text
soup = BeautifulSoup(toronto_data,"html5lib")

#find a html table in the web page
table = soup.find('table') # in html table is represented by the tag <table>


#Get all rows from the table
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [42]:
#Toronto Dataframe
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [61]:
#latitude and the longitude coordinates of each neighborhood

!wget -O Geoespatial_coordinates.csv https://raw.githubusercontent.com/josebonillads/Coursera_Capstone/main/Geospatial_Coordinates.csv

print('Data downloaded!')


--2021-05-14 20:07:28--  https://raw.githubusercontent.com/josebonillads/Coursera_Capstone/main/Geospatial_Coordinates.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2892 (2.8K) [text/plain]
Saving to: ‘Geoespatial_coordinates.csv’

Geoespatial_coordin 100%[===================>]   2.82K  --.-KB/s    in 0s      

2021-05-14 20:07:28 (20.3 MB/s) - ‘Geoespatial_coordinates.csv’ saved [2892/2892]

Data downloaded!


In [63]:
#Create geodataframe for merging
df1 = pd.read_csv('Geoespatial_coordinates.csv',error_bad_lines=False)

# take a look at the dataset
df1.head()




,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [67]:
toronto_data_df = df.merge(df1, how='inner', on='PostalCode')
toronto_data_df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
